<a href="https://colab.research.google.com/github/AshwinSinha-2004/Fake-News-Generator-Detector-Generative-AI/blob/main/Fake_News_Generator_%26_Detector_using_Generative_AI_%26_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch scikit-learn nltk pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import torch
from transformers import (
    GPT2LMHeadModel, GPT2Tokenizer,
    BertForSequenceClassification, BertTokenizer
)
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import re
import nltk
from nltk.corpus import stopwords

# Download NLTK resources
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Load datasets
real_df = pd.read_csv('True_News.csv')
fake_df = pd.read_csv('Fake_News.csv')

# Extract and preprocess headlines
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', str(text))
    text = text.lower()
    return text

real_headlines = real_df['title'].apply(preprocess_text).tolist()
fake_headlines = fake_df['title'].apply(preprocess_text).tolist()

# Split data
real_train, real_test = train_test_split(real_headlines, test_size=0.2, random_state=42)
fake_train, fake_test = train_test_split(fake_headlines, test_size=0.2, random_state=42)

In [ ]:
class HeadlineGenerator:
    def __init__(self, model_name="gpt2"):
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = GPT2LMHeadModel.from_pretrained(model_name).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

    def train(self, headlines, epochs=2, batch_size=8, lr=5e-5):
        self.model.train()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)

        encodings = self.tokenizer(
            headlines,
            truncation=True,
            padding=True,
            max_length=64,
            return_tensors="pt"
        )

        dataset = torch.utils.data.TensorDataset(
            encodings['input_ids'],
            encodings['attention_mask'],
            encodings['input_ids']
        )
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        for epoch in range(epochs):
            epoch_loss = 0
            for inputs, mask, labels in loader:
                inputs = inputs.to(self.model.device)
                mask = mask.to(self.model.device)
                labels = labels.to(self.model.device)

                optimizer.zero_grad()
                outputs = self.model(inputs, attention_mask=mask, labels=labels)
                loss = outputs.loss
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            print(f"Epoch {epoch+1}/{epochs} Loss: {epoch_loss/len(loader):.4f}")

    def generate(self, prompt="", max_length=25, num_samples=5, temperature=0.9):
        self.model.eval()
        generated = []

        with torch.no_grad():
            inputs = self.tokenizer.encode(prompt, return_tensors="pt").to(self.model.device)
            outputs = self.model.generate(
                inputs,
                max_length=max_length,
                num_return_sequences=num_samples,
                temperature=temperature,
                top_p=0.9,
                repetition_penalty=1.2,
                pad_token_id=self.tokenizer.eos_token_id,
                do_sample=True
            )
            for output in outputs:
                text = self.tokenizer.decode(output, skip_special_tokens=True)
                text = re.sub(r'\s+', ' ', text).strip()
                text = text.split('.')[0]
                generated.append(text)
        return generated

In [ ]:
# Train generator
print("Training Fake News Generator...")
generator = HeadlineGenerator()
generator.train(real_train, epochs=2, batch_size=4)

# Generate headlines
generated_headlines = generator.generate(
    prompt="Breaking news:",
    max_length=25,
    num_samples=5,
    temperature=0.95
)

Training Fake News Generator...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch 1/2 Loss: 3.4458


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Epoch 2/2 Loss: 2.4033


In [ ]:
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=64):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding=True,
            max_length=max_length,
            return_tensors="pt"
        )
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)

In [ ]:
class FakeNewsDetector:
    def __init__(self, model_name="bert-base-uncased"):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertForSequenceClassification.from_pretrained(
            model_name,
            num_labels=2
        ).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

    def prepare_data(self, real_headlines, fake_headlines):
        texts = real_headlines + fake_headlines
        labels = [0] * len(real_headlines) + [1] * len(fake_headlines)

        X_train, X_val, y_train, y_val = train_test_split(
            texts, labels, test_size=0.2, random_state=42
        )

        train_dataset = NewsDataset(X_train, y_train, self.tokenizer)
        val_dataset = NewsDataset(X_val, y_val, self.tokenizer)

        train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=16)

        return train_loader, val_loader

    def train(self, train_loader, val_loader, epochs=2, lr=2e-5):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.model.train()

        for epoch in range(epochs):
            epoch_loss = 0
            for batch in train_loader:
                inputs = {k: v.to(self.model.device) for k, v in batch.items()}
                outputs = self.model(**inputs)
                loss = outputs.loss
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                epoch_loss += loss.item()

            val_acc = self.evaluate(val_loader)
            print(f"Epoch {epoch+1}/{epochs} | Loss: {epoch_loss/len(train_loader):.4f} | Val Acc: {val_acc:.4f}")

    def evaluate(self, data_loader):
        self.model.eval()
        predictions, true_labels = [], []

        with torch.no_grad():
            for batch in data_loader:
                inputs = {k: v.to(self.model.device) for k, v in batch.items()}
                labels = inputs.pop('labels')
                outputs = self.model(**inputs)
                logits = outputs.logits
                preds = torch.argmax(logits, dim=1)
                predictions.extend(preds.cpu().numpy())
                true_labels.extend(labels.cpu().numpy())

        return accuracy_score(true_labels, predictions)

    def predict(self, headline):
        self.model.eval()
        encoding = self.tokenizer(
            headline,
            truncation=True,
            padding=True,
            max_length=64,
            return_tensors="pt"
        ).to(self.model.device)

        with torch.no_grad():
            outputs = self.model(**encoding)
            logits = outputs.logits
            pred = torch.argmax(logits, dim=1).item()

        return pred

In [ ]:
# Train detector
print("\nTraining Fake News Detector...")
detector = FakeNewsDetector()
train_loader, val_loader = detector.prepare_data(real_train, fake_train)
detector.train(train_loader, val_loader, epochs=2)


Training Fake News Detector...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2 | Loss: 0.0182 | Val Acc: 0.9979
Epoch 2/2 | Loss: 0.0045 | Val Acc: 0.9989


In [ ]:
# Gradio GUI Implementation
import gradio as gr

# Preprocessing function for input text
def preprocess_input(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', str(text))
    text = text.lower()
    return text

# Function to generate headlines
def generate_headlines(num_headlines, prompt=""):
    processed_prompt = preprocess_input(prompt)
    num_headlines = int(num_headlines)
    headlines = generator.generate(
        prompt=processed_prompt,
        max_length=25,
        num_samples=num_headlines,
        temperature=0.95
    )
    return "\n".join([f"{i+1}. {h}" for i, h in enumerate(headlines)])

# Function to detect fake news
def detect_fake(headline):
    processed_headline = preprocess_input(headline)
    prediction = detector.predict(processed_headline)
    return "FAKE" if prediction == 1 else "REAL"

# Create the Gradio interface
with gr.Blocks(title="Fake News Generator & Detector") as demo:
    gr.Markdown("# 🗞️ Fake News Generator & Detector")

    with gr.Tab("Generate Fake News"):
        gr.Markdown("## Generate Fake News Headlines")
        num_headlines = gr.Slider(1, 10, value=5, step=1, label="Number of Headlines to Generate")
        prompt = gr.Textbox(label="Starting Prompt (Optional)", value="Breaking news:")
        generate_btn = gr.Button("Generate Headlines")
        output_headlines = gr.Textbox(label="Generated Headlines", lines=6, interactive=False)

        generate_btn.click(
            fn=generate_headlines,
            inputs=[num_headlines, prompt],
            outputs=output_headlines
        )

    with gr.Tab("Detect Fake News"):
        gr.Markdown("## Detect Fake News Headlines")
        headline_input = gr.Textbox(label="Enter Headline to Analyze")
        detect_btn = gr.Button("Check Authenticity")
        result = gr.Label(label="Detection Result")

        detect_btn.click(
            fn=detect_fake,
            inputs=headline_input,
            outputs=result
        )

    gr.Markdown("### System Information")
    gr.Markdown(f"Running on {'GPU 🔥' if torch.cuda.is_available() else 'CPU 🐢'} | "
                f"Generator: GPT-2 | Detector: BERT")

In [ ]:
# Launch the interface
print("\n=== Launching Fake News Generation & Detection System ===")
demo.launch()


=== Launching Fake News Generation & Detection System ===
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1eee83da5664d97c98.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
